# Chat Completion
You also can interact with your CodeGPT Plus agents usig the Python SDK. Here, we explain how to do it with and without streaming.

### Requirements
- Your CodeGPT Plus [API Key and Org ID](https://app.codegpt.co/es/login?next=%2Fen%2Fapikeys)
- The [Python SDK](https://github.com/JudiniLabs/judini-python): `pip install judini`

In [1]:
# SDK Installation
!pip install judini --quiet

## Initialization
To initialize CodeGPT Plus, you need to provide your API Key and Org ID. There are two ways to give this information:
1. Using environment variables `CODEGPT_API_KEY` and `CODEGPT_ORG_ID`
2. Giving the API Key and Org ID as parameters

In [3]:
from judini import CodeGPTPlus
# This will read from environment variables
codegpt_client = CodeGPTPlus()

# This explicitly sets the API key and organization ID
# import os
# CODEGPT_API_KEY = os.getenv("CODEGPT_API_KEY")
# CODEGPT_ORG_ID = os.getenv("CODEGPT_ORG_ID")
# codegpt_client = CodeGPTPlus(api_key=CODEGPT_API_KEY, org_id=CODEGPT_ORG_ID)

First, lets get an agent ID. We can do that using the `get_agents` method.

In [6]:
agents_list = codegpt_client.get_agents()
print(f"Total agents: {len(agents_list)}")
agent = agents_list[0]
print(agent.model_dump_json(indent=4))

Total agents: 1
{
    "id": "b64e718d-4b7c-420b-bc5f-345991973677",
    "name": "My agent updated",
    "prompt": "You are a helpful assistant who uses a lot of emojies",
    "model": "gpt-3.5-turbo",
    "agent_documents": null,
    "welcome": "Hello, how can I help you today?",
    "pincode": null,
    "is_public": false,
    "agent_type": "qa"
}


## Chat completion

The `chat_completion` method is used to interact with your agent. It accepts the following parameters:

- `agent_id`: The ID of the agent to chat with.
- `messages`: An array of message objects to be sent to the agent. Each
            object should have a `role` (which can be 'system', 'user',
            or 'assistant') and `content` which is the actual message.
- `stream`: Whether to stream the response or not. Defaults to False.
- `format`: The format of the response. Can be either 'json' or 'text'. Defaults to 'text'.

Let's create the `messages` array 

In [14]:
messages = [{'role': 'user', 'content': 'Write a poem about markdown'}]

### No streaming

In [10]:
response = codegpt_client.chat_completion(agent_id=agent.id, messages=messages,
                                          stream=False)
print(response)

Markdown, oh Markdown, a language so neat,
With simple syntax and formatting that's sweet.
Bold and italic, you can easily achieve,
Using double asterisks or underscores, believe!

Strikethrough, oh glorious strike,
Double tildes, it's what you like.
To show that something is no more,
Markdown makes it easy to ignore.

Code blocks, a place for coding delight,
With backticks, your code shines so bright.
Python or HTML, whatever you please,
Markdown has got you covered with ease.

Lists, both ordered and unordered,
Markdown keeps them well-ordered.
Item by item, they're displayed,
Subitems nested, perfectly laid.

Links and images, they're a breeze,
With Markdown, it's a total ease.
OpenAI's website, just a click away,
Or an image of the logo, brightening your day.

Blockquotes, for important words to say,
Markdown helps them stand out, hooray!
Quoting, emphasizing, making a point,
Markdown makes it easy, no need to disappoint.

So here's to Markdown, a language so fine,
Making text form

### Streaming

In [15]:
for chunk in codegpt_client.chat_completion(agent_id=agent.id, messages=messages,
                                            stream=True):
    print(chunk, end="")

Markdown, oh Markdown, a language so sweet,
With simple syntax, it's truly a treat.
Formatting text with just a few symbols,
Making it easy for all to be nimble.

Bold and italic, with asterisks and lines,
Emphasizing words to make them shine.
Strikethrough for things that are no more,
And code blocks for lines we adore.

Lists, both ordered and unordered,
Organizing thoughts, never disordered.
Links and images, bringing life to the page,
Connecting ideas, like a vibrant stage.

Blockquotes for wisdom, profound and deep,
Words of wisdom, forever to keep.
And code snippets, a programmer's delight,
Syntax highlighted, each line in sight.

Markdown, you simplify our creation,
Making documents a delightful sensation.
So let us embrace your simplicity,
And create with joy, for all to see.

## A more complex example

What if you want to interact in a continuous way with your agent?. You can append the agent response to the `messages` array in each iteration.

In [22]:
continuous_messages = []
while True:
    new_input = input(">> ")
    if new_input == "exit":
        break
    print(f"\n>> {new_input}")
    continuous_messages.append({'role': 'user', 'content': new_input})
    response = codegpt_client.chat_completion(agent_id=agent.id, messages=continuous_messages,
                                              stream=True)
    agent_response = ""
    for chunk in response:
        agent_response += chunk
        print(chunk, end="")
    continuous_messages.append({'role': 'assistant', 'content': agent_response})



>> Hi!
Hello! How can I assist you today?
>> tell me a joke about markdown
Sure, here's a joke about Markdown:

Why did the Markdown document go to therapy?

Because it had too many formatting issues and needed some text therapy! 😄
>> text therapy?
Yes, "text therapy" is a play on words. In the context of the joke, it refers to the idea that the Markdown document had so many formatting issues that it needed some help or therapy to fix them. "Text therapy" is not a real form of therapy, but rather a playful concept in the context of the joke.
>> nice! now translate the joke to spanish
¡Claro! Aquí tienes el chiste sobre Markdown en español:

¿Por qué fue el documento de Markdown a terapia?

¡Porque tenía demasiados problemas de formato y necesitaba un poco de terapia de texto! 😄